# Random Useless Facts API

## Executive Summary

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/iragca/pit-predictive-analysis/blob/master/notebooks/api-exploration/1.0-useless-facts.ipynb)

This notebook explores the [Random Useless Facts](https://uselessfacts.jsph.pl/?brid=FnbvqLtDwNh68txuPkB5Bw) API and builds a simple wrapper for it.


## Table of Contents

```bash
.
├── Title Section
│       ├── Executive Summary
│       ├── Table of Contents                   #  ⭐ You are here
│       └── Setup                               #  Libraries and dataset imports
├── Building the wrapper
└── Using the wrapper
```


## Setup


In [7]:
try:
    import google.colab  # type: ignore
    IS_COLAB_ENVIRONMENT = True
except ImportError:
    IS_COLAB_ENVIRONMENT = False

if IS_COLAB_ENVIRONMENT:
    %pip install pydantic
    %pip install iragca

In [8]:
import httpx
from urllib.parse import urljoin, urlencode
from enum import Enum
from pydantic import BaseModel

# Building the wrapper


In [9]:
class Language(Enum):
    """
    Enumeration of supported languages for the Random Useless Facts API.

    Each enum member corresponds to the language code expected by the
    remote API.
    """
    ENGLISH = "en"
    SPANISH = "es"
    FRENCH = "fr"
    GERMAN = "de"
    ITALIAN = "it"

class Response(BaseModel):
    """
    Data model representing a response returned by the Random Useless Facts API.

    Parameters
    ----------
    id : str
        Unique identifier of the fact.
    text : str
        The textual content of the fact.
    source : str
        Origin or reference for the fact.
    source_url : str
        URL pointing to the source.
    language : Language
        Language in which the fact is written.
    permalink : str
        Permanent URL for accessing the fact.

    Notes
    -----
    This model is parsed from JSON using Pydantic and ensures type validation
    of the incoming API response.
    """
    id: str
    text: str
    source: str
    source_url: str
    language: Language
    permalink: str


class RandomUselessFactAPI:
    """
    Client for interacting with the Random Useless Facts API.

    Parameters
    ----------
    base_url : str, optional
        Base URL of the API. Defaults to ``"https://uselessfacts.jsph.pl/"``.

    Notes
    -----
    This class exposes convenience methods to construct API URLs and retrieve
    random facts in various languages. Requests are executed using ``httpx``.
    """
    def __init__(self, base_url: str ="https://uselessfacts.jsph.pl/") -> None:
        self.base_url = base_url

    def get_random_fact(self, language: Language = Language.ENGLISH) -> Response:
        """
        Retrieve a random useless fact.

        Parameters
        ----------
        language : Language, optional
            Language in which the fact should be returned. Defaults to
            ``Language.ENGLISH``.

        Returns
        -------
        Response
            Parsed API response containing the random fact.

        Raises
        ------
        httpx.HTTPError
            If the request fails or returns an error status code.

        Examples
        --------
        >>> api = RandomUselessFactAPI()
        >>> fact = api.get_random_fact()
        >>> fact.text
        'Banging your head against a wall for one hour burns 150 calories.'
        """
        url = self._build_random_fact_url(language)
        response = httpx.get(url)
        response.raise_for_status()
        data = response.json()

        return Response(**data)
    
    def _build_random_fact_url(self, language: Language = Language.ENGLISH) -> str:
        """
        Construct the request URL for fetching a random fact.

        Parameters
        ----------
        language : Language, optional
            Language code to include as a query parameter. Defaults to
            ``Language.ENGLISH``.

        Returns
        -------
        str
            Fully constructed URL pointing to the random fact endpoint.

        Notes
        -----
        This method is internal and is not intended to be called directly by
        users. It performs safe URL joining and query parameter encoding.
        """
        ENDPOINT = "api/v2/facts/random"
        path = urljoin(self.base_url, ENDPOINT)
        params = urlencode({"language": language.value})
        
        if params:
            return f"{path}?{params}"
        return path
    

# Using the wrapper


In [10]:
api = RandomUselessFactAPI()


In [13]:
fact: Response = api.get_random_fact(language=Language.ENGLISH)
print(fact.text)

The Black Widow spider eats her mate during or after sex.


In [14]:
fact

Response(id='c825b6201944a99734d34dd13e01b83b', text='The Black Widow spider eats her mate during or after sex.', source='djtech.net', source_url='http://www.djtech.net/humor/useless_facts.htm', language=<Language.ENGLISH: 'en'>, permalink='https://uselessfacts.jsph.pl/api/v2/facts/c825b6201944a99734d34dd13e01b83b')